In [1]:
import os
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from scipy.sparse import *
from collections import Counter
from sklearn.utils import shuffle

from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR

In [2]:
w2v = Word2Vec.load('../file/w2v.model')

In [3]:
train = pd.read_json('../file/train.json', encoding='utf-8')
test = pd.read_json('../file/val.json', encoding='utf-8')

In [4]:
train = pd.read_pickle('../file/train_pos.pickle')
test = pd.read_pickle('../file/validataion_pos.pickle')

In [5]:
# train.tags + train.ta

In [6]:
train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,plyst_title_pos,genre,genre_kor
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[여행, 음악]","[(POP, 10), (록, 10), (메탈, 10), (포크, 6), (얼터너티브...","[POP, 록, 메탈, 포크, 얼터너티브팝]"
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,[요즘],"[(록, 16), ('10-, 16), (발라드, 11), (메탈, 11), ('0...","[록, '10-, 발라드, 메탈, '00]"
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[편하, 잔잔]","[(R&B, 25), ('10-, 18), (Soul, 15), (인디음악, 12)...","[R&B, '10-, Soul, 인디음악, 록]"


In [7]:
train['all_word'] = train.tags + train.plyst_title_pos + train.genre_kor 

In [8]:
test['all_word'] = test.tags + test.plyst_title_pos + test.genre_kor 

In [9]:
w2v = Word2Vec(train.all_word, size = 500)

In [10]:
train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,plyst_title_pos,genre,genre_kor,all_word
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[여행, 음악]","[(POP, 10), (록, 10), (메탈, 10), (포크, 6), (얼터너티브...","[POP, 록, 메탈, 포크, 얼터너티브팝]","[락, 여행, 음악, POP, 록, 메탈, 포크, 얼터너티브팝]"
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,[요즘],"[(록, 16), ('10-, 16), (발라드, 11), (메탈, 11), ('0...","[록, '10-, 발라드, 메탈, '00]","[추억, 회상, 요즘, 록, '10-, 발라드, 메탈, '00]"
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[편하, 잔잔]","[(R&B, 25), ('10-, 18), (Soul, 15), (인디음악, 12)...","[R&B, '10-, Soul, 인디음악, 록]","[까페, 잔잔한, 편하, 잔잔, R&B, '10-, Soul, 인디음악, 록]"


In [11]:
train['all_word'] = train.tags + train.genre_kor
test['all_word'] = test.tags + test.genre_kor

In [12]:
def get_wv_mean(word_list):
    wv_mean = []
    for w in word_list:
        try:
            wv_mean.append(np.array(w2v.wv[w]))
        except Exception as e:
            pass
    if not wv_mean:
        return []
    return np.array(wv_mean).mean(axis=0)

In [13]:
# w2v.most_similar(['락', '여행', '음악', 'POP', '록', '메탈', '포크', '얼터너티브팝'])

In [14]:
train['vectored'] = train.all_word.apply(lambda e: get_wv_mean(e))

In [15]:
test['vectored'] = test.all_word.apply(lambda e: get_wv_mean(e))

In [16]:
song_count = dict(Counter([__ for _ in train.songs for __ in _]))
len(song_count)

615142

In [17]:
tag_count = dict(Counter([__ for _ in train.tags for __ in _]))
len(tag_count)

29160

In [18]:
sorted_songs = sorted(song_count.items(), key=lambda x: -x[1])
top_songs   = [x[0] for x in sorted_songs[:1000]]
# top_songs

In [19]:
sorted_tag = sorted(tag_count.items(), key=lambda x: -x[1])
top_tags   = [x[0] for x in sorted_tag[:100]]
# top_tags

In [20]:
# train test split
songs_train = train.songs.tolist()
songs_test = test.songs.tolist()

tags_train = train.tags.tolist()
tags_test  = test.tags.tolist()

w2v_train = train.vectored.tolist()
w2v_test = test.vectored.tolist()

ids_test   = test.id.tolist()

print(f"train set: {len(songs_train)}, test set: {len(songs_test)}")

train set: 115071, test set: 23015


In [21]:
# vectorizor

from itertools import groupby
vectored_train    = []
vectored_item_ids = {}
vectored_tags     = {}
idx = 0

for i, l in enumerate(songs_train):
    view = l
    for item_id in view:
        if item_id not in vectored_item_ids:
            vectored_item_ids[item_id] = idx
            idx += 1
    view = [vectored_item_ids[x] for x in view]
    vectored_train.append(view)

n_items = 0
for i, tags in enumerate(tags_train):
    for tag in tags:
        if tag not in vectored_tags:
            vectored_tags[tag] = n_items + idx
            idx += 1
    vectored_train[i].extend([vectored_tags[x] for x in tags])
n_items = len(vectored_item_ids)
n_tags = len(vectored_tags)

print(f"number of item: {n_items}, number of tags: {n_tags}")

number of item: 615142, number of tags: 29160


In [22]:
for i, wv_trn in enumerate(w2v_train):
    vectored_train[i].extend(wv_trn)

In [23]:
# In[7]:


vectored_test = []

for i, l in enumerate(songs_test):
    view = l
    ret = []
    for item_id in view:
        if item_id not in vectored_item_ids:
            continue
        ret.append(vectored_item_ids[item_id])
    vectored_test.append(ret)

for i, tags in enumerate(tags_test):
    ret = []
    for tag in tags:
        if tag not in vectored_tags:
            continue
        ret.append(tag)
    vectored_test[i].extend([vectored_tags[x] for x in ret])

In [24]:
for i, wv_tst in enumerate(w2v_test):
    vectored_test[i].extend(wv_tst)


In [25]:
# In[8]:


vectored_train = shuffle(vectored_train)
vectored_item_ids = {x:y for(y,x) in vectored_item_ids.items()}
vectored_tags = {(x - n_items):y for(y,x) in vectored_tags.items()}

In [26]:
# In[9]:


from scipy.sparse import csr_matrix

row, col, value = [],[],[]
for i, n in enumerate(vectored_train):
    row.extend([i]*len(n))
    col.extend(np.arange(len(n)))
    value.extend(n)

csr_train = csr_matrix((value, (row, col)))

In [27]:
# In[10]:


row, col, value = [],[],[]
for i, n in enumerate(vectored_train):
    row.extend([i]*len(n))
    col.extend(np.arange(len(n)))
    value.extend(n)

csr_test  = csr_matrix((value, (row, col)))

In [28]:
# In[11]:


from scipy.sparse import vstack
csr_whole = vstack([csr_test, csr_train])
csr_whole = csr_matrix(csr_whole)

In [29]:
# In[13]:


als_model = ALS(factors=128, regularization=0.08)
als_model.fit(csr_whole.T * 15.0)

In [30]:
# In[14]:


item_model = ALS(use_gpu=False)
tag_model = ALS(use_gpu=False)
item_model.user_factors = als_model.user_factors
tag_model.user_factors = als_model.user_factors

In [31]:
# In[15]:


item_model.item_factors = als_model.item_factors[:n_items]
tag_model.item_factors = als_model.item_factors[n_items:]

item_rec_csr = csr_train[:, :n_items]
tag_rec_csr = csr_train[:, n_items:]

In [32]:
# In[16]:


item_ret = []
tag_ret = []
from tqdm.auto import tqdm
for u in tqdm(range(csr_test.shape[0])):
    item_rec = item_model.recommend(u, item_rec_csr, N=250)
    item_rec = [vectored_item_ids[x[0]] for x in item_rec]
    tag_rec = tag_model.recommend(u, tag_rec_csr, N=250)
    tag_rec = [vectored_tags[x[0]] for x in tag_rec if x[0] in vectored_tags]
    item_ret.append(item_rec)
    tag_ret.append(tag_rec)

In [33]:
# In[18]:


tag_model.item_factors

array([], shape=(0, 128), dtype=float32)

In [34]:
# In[19]:


returnval = []
for _id, rec, base, tag_rec, tag_base in zip(ids_test, item_ret, songs_train, tag_ret, tags_train):
    rec = [r for r in rec if r not in base][:100]
    tags = [r for r in tag_rec if r not in tag_base][:10]
    if len(rec) != 100:
        rec += [t for t in top_songs if (t not in base) and (t not in rec)][:(100 - len(rec))]
    if len(tags) != 10:
        tags += [t for t in top_tags if (t not in tag_base) and (t not in tags)][:(10 - len(tags))]
    returnval.append({
        "id": _id,
        "songs": rec,
        "tags": tags
    })

In [35]:
import json
with open('results.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))